In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
import ast

In [ ]:
import re  # Importando a biblioteca de expressões regulares

In [ ]:
!pip install openai==0.28

In [ ]:
import openai

In [ ]:
openai.api_key = "Chave de acesso"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('caminho/base.csv')

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
df_filtrado = df[df['date'].dt.year >= 2021]

In [ ]:
df = df_filtrado

In [ ]:
df

In [ ]:
df['gptpt'] = ''

In [ ]:
df

In [ ]:
df['descricaoGptpt'] = ''

In [ ]:
df

In [ ]:
df['erro'] = ''

In [ ]:
df

In [ ]:
def get_completion(prompt, model):
  try:
    # formatar o prompt como uma mensagem de usuário para a API de chat
    messages = [{"role": "user", "content": prompt}]

    # chamar a API de chat com o modelo especificado, as mensagens e a temperatura
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    # retornar o conteúdo da resposta
    return response.choices[0].message["content"]
  except Exception as e:
        return "erro"

# **PROMPT completo**
Você é um profissional de marketing da empresa '" + empresa + "'. Classifique a avaliação do produto '"+ produto +"' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto '" + produto + "'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON:
{
  "classe": "classe",
  "explicacao_modelo": "string"
}

Exemplos:

Exemplo 1:

Você é um profissional de marketing da empresa 'spaten'. Classifique a avaliação do produto 'cerveja spaten' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja spaten'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'A cerveja spaten é boa, mas ela deixa um gosto muito amargo no final, isso é horrivel' JSON esperado: { "classe": "negativa", "explicacao_modelo": "Embora a avaliação comece de forma positiva, a menção ao gosto amargo no final e a expressão horrível indicam uma experiência negativa. Isso caracteriza uma mudança de polaridade (polarity shift)." }

Exemplo 2:

Você é um profissional de marketing da empresa 'Skol'. Classifique a avaliação do produto 'cerveja skol' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja skol'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'A Skol está fazendo cervejas muito ruins.' JSON esperado: { "classe": "negativa", "explicacao_modelo": "A avaliação é claramente negativa, pois afirma que a Skol está produzindo cervejas ruins." }

Exemplo 3:

Você é um profissional de marketing da empresa 'Itaipava'. Classifique a avaliação do produto 'cerveja itaipava' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja itaipava'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'Graças a ótima cerveja da Itaipava, estou com dor de barriga.' JSON esperado: { "classe": "negativa", "explicacao_modelo": "A expressão ótima cerveja é usada de forma irônica, pois o resultado foi uma dor de barriga, o que indica uma experiência negativa." }

Exemplo 4: Você é um profissional de marketing da empresa 'Brahma'. Classifique a avaliação do produto 'brahma duplo malte' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'brahma duplo malte'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'Essa cerveja da Brahma é muito boa' JSON esperado: { "classe": "positiva", "explicacao_modelo": "A avaliação expressa uma opinião positiva clara ao afirmar que a cerveja é muito boa." }

Exemplo 5: Você é um profissional de marketing da empresa 'Heineken'. Classifique a avaliação do produto 'cerveja Heineken' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja Heineken'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'Estou tomando uma Skol, mas sonhando com uma Heineken' JSON esperado: { "classe": "positiva", "explicacao_modelo": "A avaliação sugere que a Heineken é preferida em relação à Skol, o que é positivo para a Heineken." }

Exemplo 6: Você é um profissional de marketing da empresa 'Heineken'. Classifique a avaliação do produto 'cerveja Heineken' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto ' cervejaHeineken'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'a Heineken foi entregue' JSON esperado: { "classe": "neutra", "explicacao_modelo": "A avaliação apenas relata a entrega do produto, sem expressar nenhuma opinião positiva ou negativa." }

Avaliação a ser classificada:
avaliação: '" + avaliacao + "
JSON esperado:
{
  "classe": "classe",
  "explicacao_modelo": "string"
}

In [ ]:
prompt = """
Exemplos:

Exemplo 1:

Você é um profissional de marketing da empresa 'spaten'. Classifique a avaliação do produto 'cerveja spaten' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja spaten'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'A cerveja spaten é boa, mas ela deixa um gosto muito amargo no final, isso é horrivel' JSON esperado: { "classe": "negativa", "explicacao_modelo": "Embora a avaliação comece de forma positiva, a menção ao gosto amargo no final e a expressão horrível indicam uma experiência negativa. Isso caracteriza uma mudança de polaridade (polarity shift)." }

Exemplo 2:

Você é um profissional de marketing da empresa 'Skol'. Classifique a avaliação do produto 'cerveja skol' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja skol'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'A Skol está fazendo cervejas muito ruins.' JSON esperado: { "classe": "negativa", "explicacao_modelo": "A avaliação é claramente negativa, pois afirma que a Skol está produzindo cervejas ruins." }

Exemplo 3:

Você é um profissional de marketing da empresa 'Itaipava'. Classifique a avaliação do produto 'cerveja itaipava' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja itaipava'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}

avaliação: 'Graças a ótima cerveja da Itaipava, estou com dor de barriga.' JSON esperado: { "classe": "negativa", "explicacao_modelo": "A expressão ótima cerveja é usada de forma irônica, pois o resultado foi uma dor de barriga, o que indica uma experiência negativa." }

Exemplo 4: Você é um profissional de marketing da empresa 'Brahma'. Classifique a avaliação do produto 'brahma duplo malte' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'brahma duplo malte'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'Essa cerveja da Brahma é muito boa' JSON esperado: { "classe": "positiva", "explicacao_modelo": "A avaliação expressa uma opinião positiva clara ao afirmar que a cerveja é muito boa." }

Exemplo 5: Você é um profissional de marketing da empresa 'Heineken'. Classifique a avaliação do produto 'cerveja Heineken' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto 'cerveja Heineken'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'Estou tomando uma Skol, mas sonhando com uma Heineken' JSON esperado: { "classe": "positiva", "explicacao_modelo": "A avaliação sugere que a Heineken é preferida em relação à Skol, o que é positivo para a Heineken." }

Exemplo 6: Você é um profissional de marketing da empresa 'Heineken'. Classifique a avaliação do produto 'cerveja Heineken' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto ' cervejaHeineken'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'}
avaliação: 'a Heineken foi entregue' JSON esperado: { "classe": "neutra", "explicacao_modelo": "A avaliação apenas relata a entrega do produto, sem expressar nenhuma opinião positiva ou negativa." }
"""


In [ ]:
df

In [ ]:
model="gpt-3.5-turbo"

for index, row in df.iterrows():
  try:
        avaliacao = row['review']
        produto = row['Produto']
        empresa = row['Marca']

        inicio_prompt = "Você é um profissional de marketing da empresa '" + empresa + "'. Classifique a avaliação do produto '"+ produto +"' como 'positiva', 'negativa' ou 'neutra'. A avaliação pode conter sarcasmo, ironia ou polarity shift. A avaliação deve ser analisada do ponto de vista do produto '" + produto + "'. Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON:"\
        "{" \
                   " \"""classe\""": \"""classe\"""," \
                    " \"""explicacao_modelo\""": \"""string\"""" \
                "}"

        prompt_final = inicio_prompt + prompt + " Avaliação a ser classificada: "\
        "avaliação: '" + avaliacao + "' "\
                "JSON esperado:"\
                "{" \
                   " \"""classe\""": \"""classe\"""," \
                    " \"""explicacao_modelo\""": \"""string\"""" \
                "}"

        print(prompt_final)

        response = get_completion(prompt_final, model)

        print(response)

        response_dict = json.loads(response)

        # Salve o sentimento na coluna correspondente
        df.at[index, 'gptpt'] = response_dict['classe']
        df.at[index, 'descricaoGptpt'] = response_dict['explicacao_modelo'].replace("'", "")

        print("----------------------------------------------------------------------------------------------------------------------------------")
  except Exception as e:
      df.at[index, 'erro'] = response

In [ ]:
df

In [ ]:
print(df.erro.value_counts())

In [ ]:
df.to_csv('caminho/base.csv', index=False)

In [ ]:
print(df.gptpt.value_counts())

In [ ]:
df['gptpt'] = df['gptpt'].replace({'negativa': -1, 'positiva': 1, 'neutra':0})

In [ ]:
print(df.gptpt.value_counts())

In [ ]:
df